In [32]:
import re
import sys
import os
import xml.etree.ElementTree as ET
import xml.dom.minidom
import xml
import requests
import json
from pathlib import Path
import PyPDF2 as ppdf
import string
import pickle
from tqdm import tqdm
from ratelimit import limits, sleep_and_retry

In [2]:
NAMES = sorted([
    'Amis',
    'Atayal',
    'Paiwan',
    'Bunun',
    'Puyma',
    'Rukai',
    'Tsou',
    'Saisiyat',
    'Yami',
    'Thao',
    'Kavalan',
    'Truku',
    'Sakizaya',
    'Seediq',
    'Saaroa',
    'Kanakanavu'
])

In [59]:
def getRoot(tr: str):
    # tr = tribe name with proper capitalization
    cwd = os.getcwd()
    folder = '.PanglossXML'
    file = os.path.join(cwd, folder, tr, tr + ".xml")
    data = ET.parse(file)
    root = data.getroot()
    return root

In [50]:
root = getRoot('Atayal')
urlPairs = {k.attrib['id']: k[2].attrib['url'] for k in root}

In [58]:
for tr in tqdm(NAMES):
    # file path setup + get xml file
    cwd = os.getcwd()
    folder = '.PanglossXML'
    file = os.path.join(cwd, folder, tr, tr + ".xml")
    data = ET.parse(file)
    root = data.getroot()

    # parse into dict of {xml id: url to audio}
    urlPairs = {k.attrib['id']: k[2].attrib['url'] for k in root}

    # downloading loop
    for i, (k, v) in enumerate(urlPairs.items()):
        # check
        assert root[i].attrib['id'] == k and root[i][2].attrib['url'] == v
        
        # TODO
        audioPath = os.path.join(cwd, folder, tr)
        root[i][2].attrib['path'] = os.path.relpath(file)


100%|██████████| 16/16 [00:00<00:00, 16.32it/s]
